## Práctica 2. Tema 5. Búsqueda de hiperparámetros en ML.
Aplicar hiperparámetros externos a los modelos de ML para alcanzar una predicción mejor.

#### Importamos las librerías necesarias:

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, KFold
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from random import uniform

#### Cargamos la base de datos:

In [16]:
data_df = pd.read_csv('../datasets/south_africa_chd.csv')

#### Procesamiento de los datos:

In [17]:
data_df.shape
data_df.head()
ohe_famhist = OneHotEncoder(drop='if_binary').fit_transform(data_df[['famhist']])
data_df['famhist'] = ohe_famhist.toarray()
ohe_chd = OneHotEncoder(drop='if_binary').fit_transform(data_df[['chd']])
data_df['chd'] = ohe_chd.toarray()
data_df.head()
data = data_df.to_numpy()
data.shape

(462, 10)

Dividimos los datos en conjuntos de entrenamiento, validación y prueba:
- Traning set -> 60% de los datos
- Validation set -> 20% de los datos
- Test set -> 20% de los datos

In [18]:
X = data[:,0:-1]
Y = data[:,-1]
X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=0.4, random_state=42, stratify=Y)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=42, stratify=Y_temp)

Normalizar las características:

In [19]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

Applicamos validación cruzada con K-fold:

In [20]:
kf = KFold(n_splits=5, shuffle=True, random_state=0)

## Regresión Lineal -> Método de regresión ML

#### Búsqueda de hiperparámetros para regresión lineal usando GridSearchCV:
- Grid Search: Se define un rango o conjunto de valores posibles para cada hiperparámetro, y el método prueba todas las combinaciones posibles.

In [21]:
param_grid_lr = {
    'fit_intercept': [True, False],       # Incluir o no el intercepto
    'copy_X': [True, False],              # Copiar X antes de ajustar
    'n_jobs': [-1, 1],                    # Número de trabajos para paralelización
    'positive': [True, False]             # Forzar los coeficientes a ser no negativos
}

grid_search_lr = GridSearchCV(LinearRegression(), param_grid_lr, cv=kf, scoring='neg_mean_squared_error')
grid_search_lr.fit(X_train, Y_train)

print("Mejores parámetros para Regresión Lineal (GridSearchCV):", grid_search_lr.best_params_)
print("Mejor score de validación (GridSearchCV) para regresión lineal:", grid_search_lr.best_score_)

Mejores parámetros para Regresión Lineal (GridSearchCV): {'copy_X': True, 'fit_intercept': True, 'n_jobs': -1, 'positive': True}
Mejor score de validación (GridSearchCV) para regresión lineal: -0.19098900727769477


Predicciones y evaluaciones de la Regresión Lineal con Grid Search:

In [22]:
Y_pred_lr1 = grid_search_lr.predict(X_test)
# Error cuadrático medio -> Error, en media, cuadrático de las predicciones frente a valores observados
mse_lr = mean_squared_error(Y_test, Y_pred_lr1)
# Coeficiente de determinación -> Variabilidad de las variables a predecir, (entre 0: NO puede predecir, y 1: predice perfectamente)
r2_lr = r2_score(Y_test, Y_pred_lr1) 
print(f'MSE (GridSearchCV): {mse_lr}, R2 (GridSearchCV): {r2_lr}')

MSE (GridSearchCV): 0.16600652404354585, R2 (GridSearchCV): 0.26445162579271075


Si lo comparamos con los resultados obtenidos en la anterior práctica, podemos ver que obtenemos resultados mejores. Con un error cuadrático medio menor, indicando que se comete menos errores a la hora de predecir; y un coeficiente de determinación algo mayor, pero sigue estando bastante alejado de la unidad para poder concluir que se realiza una predicción de forma correcta.

#### Búsqueda de hiperparámetros para Regresión Lineal con RandomizedSearchCV:
- Random Search: En lugar de probar todas las combinaciones posibles (como en Grid Search), selecciona un número fijo de combinaciones aleatorias.

In [23]:
param_grid_lr = {
    'fit_intercept': [True, False],       # Incluir o no el intercepto
    'copy_X': [True, False],              # Copiar X antes de ajustar
    'n_jobs': [-1, 1],                    # Número de trabajos para paralelización
    'positive': [True, False]             # Forzar los coeficientes a ser no negativos
}

random_search_lr = RandomizedSearchCV(LinearRegression(), param_grid_lr, cv=kf, scoring='neg_mean_squared_error')
random_search_lr.fit(X_train, Y_train)

print("Mejores parámetros para Regresión Lineal (GridSearchCV):", random_search_lr.best_params_)
print("Mejor score de validación (GridSearchCV) para regresión lineal:", random_search_lr.best_score_)

Mejores parámetros para Regresión Lineal (GridSearchCV): {'positive': True, 'n_jobs': -1, 'fit_intercept': True, 'copy_X': True}
Mejor score de validación (GridSearchCV) para regresión lineal: -0.19098900727769477


Predicciones y evaluciones de la Regresión Lineal con Random Search:

In [24]:
Y_pred_lr2 = random_search_lr.predict(X_test)
# Error cuadrático medio -> Error, en media, cuadrático de las predicciones frente a valores observados
mse_lr = mean_squared_error(Y_test, Y_pred_lr2)
# Coeficiente de determinación -> Variabilidad de las variables a predecir, (entre 0: NO puede predecir, y 1: predice perfectamente)
r2_lr = r2_score(Y_test, Y_pred_lr2) 
print(f'MSE (GridSearchCV): {mse_lr}, R2 (GridSearchCV): {r2_lr}')

MSE (GridSearchCV): 0.16600652404354585, R2 (GridSearchCV): 0.26445162579271075


Obtenemos mediante este método, los mismos resultados que en el método anterior, pero podríamos haber obtenido un resultado ligeramente peor. Ya que en este caso, no evaluamos todas las combinaciones como en Grid Search, sino un número de combinaciones fijas al azar.



## Regresión Logística -> Método de clasificación ML

#### Búsqueda de hiperparámetros para Regresión Logística usando GridSearchCV:
- Grid Search: Se define un rango o conjunto de valores posibles para cada hiperparámetro, y el método prueba todas las combinaciones posibles.

In [25]:
param_grid_logr = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],                        # Valores de C para regularización
    'solver': ['liblinear', 'saga', 'newton-cg', 'lbfgs'],      # Solvers disponibles
    'max_iter': [100, 200, 300, 500],                           # Número de iteraciones
}

grid_search_logr = GridSearchCV(LogisticRegression(), param_grid_logr, cv=kf, scoring='accuracy', n_jobs=-1, verbose=1)
grid_search_logr.fit(X_train, Y_train)

print("Mejores parámetros para Regresión Logística (GridSearchCV):", grid_search_logr.best_params_)
print("Mejor score de validación (GridSearchCV) para regresión logística:", grid_search_logr.best_score_)

Fitting 5 folds for each of 96 candidates, totalling 480 fits
Mejores parámetros para Regresión Logística (GridSearchCV): {'C': 0.1, 'max_iter': 100, 'solver': 'lbfgs'}
Mejor score de validación (GridSearchCV) para regresión logística: 0.725844155844156


Predicciones y evaluaciones de la Regresión Logística con Grid Search:

In [26]:
Y_pred1 = grid_search_logr.predict(X_test)

# Evaluar el rendimiento
accuracy1 = accuracy_score(Y_test, Y_pred1)
conf_matrix1 = confusion_matrix(Y_test, Y_pred1)

# Mostrar los resultados
print(f"Accuracy: {accuracy1:.4f}")
print("Matriz de Confusión:")
print(conf_matrix1)

Accuracy: 0.7204
Matriz de Confusión:
[[49 12]
 [14 18]]


#### Búsqueda de hiperparámetros para Regresión Logística con RandomizedSearchCV:
- Random Search: En lugar de probar todas las combinaciones posibles (como en Grid Search), selecciona un número fijo de combinaciones aleatorias.

In [27]:
param_dist_lr = {
    'C': uniform(0.001, 100),                               # Regularización
    'solver': ['liblinear', 'saga', 'newton-cg', 'lbfgs'],  # Solvers disponibles
    'max_iter': [300, 500, 1000],                           # Número máximo de iteraciones
}

random_search_logr = RandomizedSearchCV(LogisticRegression(), param_dist_lr, n_iter=100, cv=kf, scoring='accuracy', random_state=0)
random_search_logr.fit(X_train, Y_train)

print("Mejores parámetros para Regresión Logística (RandomizedSearchCV):", random_search_logr.best_params_)
print("Mejor score de validación (RandomizedSearchCV) para regresión logística:", random_search_logr.best_score_)

Mejores parámetros para Regresión Logística (RandomizedSearchCV): {'C': 54.88235039273247, 'max_iter': 500, 'solver': 'liblinear'}
Mejor score de validación (RandomizedSearchCV) para regresión logística: 0.7005194805194807


Predicciones y evaluciones de la Regresión Logística con Random Search:

In [28]:
Y_pred2 = random_search_logr.predict(X_test)

# Evaluar el rendimiento
accuracy2 = accuracy_score(Y_test, Y_pred2)
conf_matrix2 = confusion_matrix(Y_test, Y_pred2)

# Mostrar los resultados
print(f"Accuracy: {accuracy2:.4f}")
print("Matriz de Confusión:")
print(conf_matrix2)

Accuracy: 0.7204
Matriz de Confusión:
[[48 13]
 [13 19]]


En el método de ML de Regresión Logóstica, con ambos métodos de obtención de hiperparámetros, obtenemos los mismos resultados; los cuales no llegan a mejorar el anterior modelo sin hiperparámetros. Esto puede ser debido a la poca densidad de datos que tenemos, ya que se generar patrones distintos que antes y podría afectar a una clase concreta más que a otra, por lo que se podría ver mejoría con mucho mayor volumen de datos.